In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-16:] == 'image-to-tex-OCR':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR/Jupyter_Notebooks
Current path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR


In [2]:
# Import the Printed_Tex_Data_Module

from Data.Data_Module import Data_Module
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
from IPython.display import display, Math

In [3]:
df = Data_Module()
df.prepare_data()

In [6]:
df.raw_df

,formula,image_name,height,width,formula_length
0,\widehat { H } = \displaystyle \frac { 2 \pi }...,db69228953de400.png,64,512,289
1,D _ { \alpha } ^ { i } D ^ { j \alpha } \; w =...,a452f8cc9263922.png,64,512,135
2,s h _ { 2 } + \nu = \partial ^ { i } \left( 2 ...,b76614553695957.png,64,512,303
3,Q _ { 2 n } ( { \bf A } ) = \left\langle { \bf...,56102e6fec5ead8.png,64,512,74
4,Z _ { 2 n } = 1 - \frac { 1 } { n } \ell \delt...,efbd72c272f1c47.png,64,512,199
...,...,...,...,...,...
64,\frac { 1 } { 4 \pi } \int _ { l ^ { 4 } } [ |...,29e5c61c5d19549.png,64,512,155
65,"\vert j , m > = \frac { { a _ { + } ^ { \dagge...",3f37131cacf0642.png,64,512,159
66,{ \cal O } _ { 2 } ^ { i } { } _ { k } { \cal ...,a31f964db6eda23.png,64,512,622
67,"\int d ^ { 2 } y \, \left( V _ { ( \mu ) } ^ {...",b188007079ca183.png,64,512,166
